### Classification of AD vs MCI vs NC Using the ResNet Pre-Trained Model

Import the packages

In [4]:
! pip install -r requirements.txt

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import torch
from torch import nn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader
from torchinfo import summary
import shutil
import data_manager as DM
# import torchvision.models as models
import torchvision.models.video as models
from torchvision.models.video import R3D_18_Weights, r3d_18
import data_setup, engine
from helper_functions import plot_loss_curves
from data_setup import create_dataloaders
import engine
from monai.transforms import Resize
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import DataLoader
from helper_functions import plot_loss_curves
import torchio as tio
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
import scipy.io as sio
import monai
import nibabel as nib
from Update_Git import git_add, git_commit, git_push

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

d:\Python-codes\packages\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Error executing command: ['git', 'add', 'path_to_your_file']
Error message: fatal: pathspec 'path_to_your_file' did not match any files

Failed to add path_to_your_file to git.
Using device: cpu


## Git

Add the parrent path to current path because data is there

In [ ]:
current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

file_path = "main.ipynb"
git_add(file_path)
git_commit("Update main.ipynb")
git_push("main")

TypeError: git_push() missing 1 required positional argument: 'branch_name'

Manage data:

✔ Read subject IDs from each sheet in Subject list.xlsx.

✔ Create Data/AD, Data/MCI, Data/NC folders.

✔ Find std_T1.nii for each subject inside ADNI/{subject_id}/.

✔ Copy & renames the file to Data/{category}/{subject_id}.nii.

In [7]:
categories = ["AD", "MCI", "NC"]
excel_file = "../Subject list.xlsx"
source_root = "ADNI"
destination_root = "Data"
if os.path.join(destination_root):
   os.system('rm -r Data')
source_dir = "../"
DM.copy_data(excel_file,source_root,source_dir, destination_root,categories)

FileNotFoundError: [Errno 2] No such file or directory: '../Subject list.xlsx'

How many subjects do we have in each group?

In [ ]:
data_root = "Data"

for c in categories:
    path_train = os.path.join(data_root, 'train', c)
    path_test = os.path.join(data_root, 'test', c)

    num_train_files = len(os.listdir(path_train))
    print(f"{c} train: {num_train_files} files")

    num_test_files = len(os.listdir(path_test))
    print(f"{c} test: {num_test_files} files")


## Data loader: Prepare the data for model training and testing

In [ ]:
# How is the ViT default transformation?
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT
pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)
transform = pretrained_vit_weights.transforms()
print(transform)

In [ ]:
# Define the batch size
batch_size = 16

# pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT
# pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)
# transform = pretrained_vit_weights.transforms()
# print(transform)

transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=InterpolationMode.BILINEAR),
    transforms.Normalize(mean=[0.485], std=[0.229])
])
print(f"transform: {transform}")

# Augmentaed dataset to be added to the main dataset
augmentation_transform = transforms.Compose([
    #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust colors
    transforms.RandomHorizontalFlip(p=1),  # Random horizontal flip
    transforms.RandomVerticalFlip(p=0.3),  # Random vertical flip
    transforms.RandomRotation(degrees=15, interpolation=InterpolationMode.BICUBIC),  # Random rotation
    transforms.Resize(size=256, interpolation=InterpolationMode.BILINEAR),  # Resize to 256
    transforms.CenterCrop(size=224),  # Center crop to 224x224
    #transforms.ToTensor(),  # Convert image to tensor

])


# Data loader
train_data_path = os.path.join(data_root,"train")
test_data_path = os.path.join(data_root,"test")

train_dataloader_pretrained, test_dataloader_pretrained, class_names = data_setup.create_dataloaders(
    train_dir=train_data_path, 
    test_dir=test_data_path,
    transform=transform,
    batch_size=batch_size,
    # augmentation_transform=augmentation_transform
)

print(' ')
print(f"Class names: {class_names}")
print(f"Number of classes: {len(class_names)}")

print(' ')
print("Number of training data: ", len(train_dataloader_pretrained) * batch_size)
print("Number of testing data: ", len(test_dataloader_pretrained) * batch_size)

image_batch, label_batch = next(iter(train_dataloader_pretrained))
print(image_batch.shape, label_batch.shape)


## Plot

In [ ]:
# Select the first 9 images (first slice of each)
num_images = 6
image_batch_np = image_batch.cpu()  # Move to CPU

fig, axes = plt.subplots(2, 3, figsize=(9, 9))

for i, ax in enumerate(axes.flat):  
    img = image_batch_np[i, 30]  # Take one slice (shape: 1, 224, 224)
    img_np = img.numpy().transpose(1, 2, 0)  # Convert from (C, H, W) → (H, W, C)
    ax.imshow(img_np)  # Show RGB image
    ax.axis("off")
    ax.set_title(f"Label: {class_names[label_batch[i].item()]}")

plt.tight_layout()
plt.show()


## Classification Model

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the pretrained ViT model
pretrained_vit_weights = torchvision.models.ViT_B_16_Weights.DEFAULT
pretrained_vit = torchvision.models.vit_b_16(weights=pretrained_vit_weights).to(device)

# Freeze pretrained weights
for parameter in pretrained_vit.parameters():
    parameter.requires_grad = False

# Modify the conv_proj to accept 3-channel images (RGB slices)
pretrained_vit.conv_proj = nn.Conv2d(
    in_channels=1,  # 3-channel input (RGB slices)
    out_channels=768,  # Output channels must match ViT embeddings
    kernel_size=16,
    stride=16
).to(device)

# Remove the classification head as we'll define our own
pretrained_vit.heads = nn.Identity()

# Define the ViT3D model with aggregation
class ViT3D(nn.Module):
    def __init__(self, vit_model, num_slices=79, num_classes=3, aggregation="mean"):
        super(ViT3D, self).__init__()
        self.vit = vit_model
        self.num_slices = num_slices
        self.aggregation = aggregation  # "mean", "max" for feature aggregation
        
        # Fully connected classification head
        self.fc = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(768, num_classes)  # Output dimension matches the number of classes
        )
    
    def forward(self, x):
        batch_size, num_slices, C, H, W = x.shape  # Shape: (B, 79, 3, 224, 224)
        x = x.view(batch_size * num_slices, C, H, W)  # Flatten slices: (B*79, 3, 224, 224)
        
        features = self.vit(x)  # Shape: (B*79, 768) after passing through ViT
        
        features = features.view(batch_size, num_slices, -1)  # Reshape to (B, 79, 768)

        # Aggregate features across slices using specified method
        if self.aggregation == "mean":
            features = torch.mean(features, dim=1)  # Mean pooling across slices
        elif self.aggregation == "max":
            features, _ = torch.max(features, dim=1)  # Max pooling across slices
        else:
            raise ValueError("Invalid aggregation method. Choose 'mean' or 'max'.")

        return self.fc(features)  # Final classification step
    
# Initialize the ViT3D model
vit3d_model = ViT3D(pretrained_vit, num_slices=79, num_classes=3, aggregation="mean").to(device)

# Print the model summary
summary(model=vit3d_model,
        input_size=(8, 79, 1, 224, 224),  # Input shape: (batch_size, slices, channels, height, width)
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)


## TRAIN AND TEST

In [ ]:
# Check the device
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Working on device: {device}")

# Create optimizer and loss function
optimizer = torch.optim.Adam(vit3d_model.parameters(), lr=1e-64, weight_decay=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Train the classifier head of the pretrained ViT feature extractor model
torch.manual_seed(42)
torch.cuda.manual_seed(42)

pretrained_RN_18_results = engine.train(
    model=vit3d_model,
    train_dataloader=train_dataloader_pretrained,
    test_dataloader=test_dataloader_pretrained,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=5,
    device=device
)


## PLOT

In [ ]:
plot_loss_curves(vit3d_model)